In [5]:
import torch
import torchvision.transforms as tvt
from torch.optim import Adam

from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10
from tqdm.notebook import tqdm
from torchmetrics import Accuracy, AUROC
from torch.optim.lr_scheduler import CosineAnnealingLR

from oodtk.utils import is_known, is_unknown, contains_known, contains_unknown
from oodtk.loss import ObjectosphereLoss
from oodtk.dataset.img import Textures, CIFAR10C, CIFAR10P, LSUNCrop, LSUNResize, TinyImageNetResize, TinyImageNetCrop
from oodtk.dataset.img import TinyImages300k
from oodtk.model import WideResNet
from oodtk.transforms import ToRGB, ToUnknown

In [6]:
torch.manual_seed(123)

mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

trans = tvt.Compose([ToRGB(), tvt.Resize((32,32)), tvt.ToTensor(), tvt.Normalize(mean, std)])

# setup data
dataset_in_train = CIFAR10(root="data", train=True, download=True, transform=trans)
# dataset_in_test = CIFAR10(root="data", train=False, transform=trans)
dataset_out_train = Subset(
    TinyImages300k(root="/home/ki/datasets/", transform=trans, target_transform=ToUnknown(), download=True), indices=range(120000))

train_loader = DataLoader(dataset_in_train + dataset_out_train, batch_size=128, shuffle=True)


Files already downloaded and verified


In [7]:
model = WideResNet.from_pretrained("cifar10-pt", num_classes=10)
opti = Adam(params=model.parameters(), lr=0.001)
crit = ObjectosphereLoss()
scheduler = CosineAnnealingLR(opti, T_max=len(train_loader) * 10)
model = model.cuda()

In [8]:

acc = Accuracy(num_classes=10).cuda()
auroc = AUROC(num_classes=2).cuda()

for epoch in range(10):
    bar = tqdm(train_loader)
    for batch in bar:
        x, y = batch
        x, y = x.cuda(), y.cuda()
        z = model(x)

        loss = crit(z, y)
        opti.zero_grad()
        loss.backward()
        opti.step()
        scheduler.step()

        acc.update(z[is_known(y)],y[is_known(y)])
        auroc.update(ObjectosphereLoss.score(z),is_unknown(y))
        bar.set_postfix({"loss": loss.item(), "acc": acc.compute().item(), "auroc": auroc.compute().item()})

    acc.reset()
    auroc.reset()

/home/ki/projects/oodtk/venv/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


  0%|          | 0/1329 [00:00<?, ?it/s]

  0%|          | 0/1329 [00:00<?, ?it/s]

  0%|          | 0/1329 [00:00<?, ?it/s]

  0%|          | 0/1329 [00:00<?, ?it/s]

...

  0%|          | 0/1329 [00:00<?, ?it/s]

  0%|          | 0/1329 [00:00<?, ?it/s]

  0%|          | 0/1329 [00:00<?, ?it/s]

  0%|          | 0/1329 [00:00<?, ?it/s]

  0%|          | 0/1329 [00:00<?, ?it/s]

In [18]:
dataset_in_test = CIFAR10(root="data", train=False, transform=trans)
dataset_out_test1 = Textures(root="data", download=True, transform=trans)
dataset_out_test2 = LSUNCrop(root="data", download=True, transform=trans)
dataset_out_test3 = LSUNResize(root="data", download=True, transform=trans)
dataset_out_test4 = TinyImageNetResize(root="data", download=True, transform=trans)
dataset_out_test5 = TinyImageNetCrop(root="data", download=True, transform=trans)
dataset_test = dataset_in_test + dataset_out_test1 + dataset_out_test2 + dataset_out_test3 + dataset_out_test4 + dataset_out_test5
test_loader = DataLoader(dataset_test, batch_size=128, shuffle=True)

acc = Accuracy(num_classes=10).cuda()
auroc = AUROC(num_classes=2).cuda()

with torch.no_grad():
    for batch in tqdm(test_loader):
        x, y = batch
        x, y = x.cuda(), y.cuda()
        z = model(x)
        acc.update(z[is_known(y)],y[is_known(y)])
        auroc.update(ObjectosphereLoss.score(z),is_unknown(y))

print(acc.compute())
print(auroc.compute())

  0%|          | 0/435 [00:00<?, ?it/s]

tensor(0.8674, device='cuda:0')
tensor(0.9585, device='cuda:0')


In [ ]:
from oodtk.dataset.img import Textures, CIFAR10C, CIFAR10P, LSUNCrop, LSUNResize, TinyImageNetResize, TinyImageNetCrop
CIFAR10P(root="data/", subset="brightness", download=True)
